In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv(r"C:\Users\dubey\Duplicate Question Pair\duplicatpairs\data\train.csv")
df.shape

(404290, 6)

In [3]:
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [9]:
new_df = df.sample(30000)

In [10]:
new_df.isnull().sum()

id              0
qid1            0
qid2            0
question1       0
question2       0
is_duplicate    0
dtype: int64

In [11]:
new_df.duplicated().sum()

0

In [12]:
ques_df = new_df[['question1','question2']]
ques_df.head()

,question1,question2
187720,What are good sources to learn Tigrinya?,What is the best way to learn Tigrinya?
77580,How can people still believe that the earth is...,Are there REALLY people who still believe the ...
375244,What is a coxswain in the Navy?,Does it matter how big a coxswain is?
119692,Will the technology be an enabler for the futu...,Will human evolution in the future be more bio...
401371,What are some good methods for data pre-proces...,Is there a methodical process to read data in ...


In [13]:
from sklearn.feature_extraction.text import CountVectorizer
# merge texts
questions = list(ques_df['question1']) + list(ques_df['question2'])

cv = CountVectorizer(max_features=3000)
q1_arr, q2_arr = np.vsplit(cv.fit_transform(questions).toarray(),2)

In [14]:
temp_df1 = pd.DataFrame(q1_arr, index= ques_df.index)
temp_df2 = pd.DataFrame(q2_arr, index= ques_df.index)
temp_df = pd.concat([temp_df1, temp_df2], axis=1)
temp_df.shape

(30000, 6000)

In [15]:
temp_df

,0,1,2,3,4,5,6,7,8,9,...,2990,2991,2992,2993,2994,2995,2996,2997,2998,2999
187720,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
77580,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
375244,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
119692,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
401371,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6399,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
106431,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
337051,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
328587,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [16]:
temp_df['is_duplicate'] = new_df['is_duplicate']

In [17]:
temp_df.head()

,0,1,2,3,4,5,6,7,8,9,...,2991,2992,2993,2994,2995,2996,2997,2998,2999,is_duplicate
187720,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
77580,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
375244,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
119692,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
401371,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [18]:

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(temp_df.iloc[:,0:-1].values,temp_df.iloc[:,-1].values,test_size=0.2,random_state=1)

In [19]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
rf = RandomForestClassifier()
rf.fit(X_train,y_train)
y_pred = rf.predict(X_test)
accuracy_score(y_test,y_pred)

0.7321666666666666

In [20]:
from xgboost import XGBClassifier
xgb = XGBClassifier()
xgb.fit(X_train,y_train)
y_pred = xgb.predict(X_test)
accuracy_score(y_test,y_pred)

0.7231666666666666